In [3]:
df =pd.read_csv(r"C:\Users\admin\Downloads\Recommendation System\Recommendation System\anime.csv")
df

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

# Load the dataset
anime_data = pd.read_csv(r"C:\Users\admin\Downloads\Recommendation System\Recommendation System\anime.csv")

# Preprocessing: Handle missing values
anime_data['genre'] = anime_data['genre'].fillna('Unknown')
anime_data['type'] = anime_data['type'].fillna('Unknown')
anime_data['rating'] = anime_data['rating'].fillna(anime_data['rating'].median())
anime_data['episodes'] = pd.to_numeric(anime_data['episodes'], errors='coerce').fillna(0).astype(int)

# Normalize numerical features
scaler = MinMaxScaler()
anime_data[['rating', 'members', 'episodes']] = scaler.fit_transform(anime_data[['rating', 'members', 'episodes']])

# Feature extraction: Multi-hot encoding for genres
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(', '))
genre_matrix = vectorizer.fit_transform(anime_data['genre'])

# Combine genre and numerical features
features = np.hstack((genre_matrix.toarray(), anime_data[['rating', 'members', 'episodes']].values))

# Compute cosine similarity
cosine_sim = cosine_similarity(features)

# Recommendation function
def recommend_anime(target_name, data, sim_matrix, top_n=5, threshold=0.7):
    try:
        target_index = data[data['name'] == target_name].index[0]
        sim_scores = list(enumerate(sim_matrix[target_index]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = [anime for anime in sim_scores if anime[1] > threshold]
        top_anime_indices = [i[0] for i in sim_scores[1:top_n+1]]
        return data.iloc[top_anime_indices][['name', 'genre', 'rating']]
    except IndexError:
        return "Anime not found."

# Example recommendation
print("Recommendations for 'Kimi no Na wa.':")
print(recommend_anime("Kimi no Na wa.", anime_data, cosine_sim))

# Evaluation
# Dummy target labels for evaluation
anime_data['dummy_label'] = (anime_data['genre'].str.contains('Action')).astype(int)
X_train, X_test, y_train, y_test = train_test_split(features, anime_data['dummy_label'], test_size=0.2, random_state=42)

# Make predictions
preds = cosine_similarity(X_test, X_train).mean(axis=1) > 0.5

# Evaluate metrics
precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Recommendations for 'Kimi no Na wa.':
                                                   name  \
5805                        Wind: A Breath of Heart OVA   
6394                       Wind: A Breath of Heart (TV)   
1111              Aura: Maryuuin Kouga Saigo no Tatakai   
504   Clannad: After Story - Mou Hitotsu no Sekai, K...   
208                       Kokoro ga Sakebitagatterunda.   

                                             genre    rating  
5805          Drama, Romance, School, Supernatural  0.561825  
6394          Drama, Romance, School, Supernatural  0.536615  
1111  Comedy, Drama, Romance, School, Supernatural  0.720288  
504                         Drama, Romance, School  0.762305  
208                         Drama, Romance, School  0.798319  
Precision: 0.00
Recall: 0.00
F1-Score: 0.00


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [1]:
# Explanation of Collaborative Filtering
print("\nCollaborative Filtering recommends items based on user interactions. Two types:")
print("1. User-Based: Finds similar users and recommends items they liked.")
print("2. Item-Based: Finds similar items and recommends based on past interactions.")


Collaborative Filtering recommends items based on user interactions. Two types:
1. User-Based: Finds similar users and recommends items they liked.
2. Item-Based: Finds similar items and recommends based on past interactions.
